# window_size = 5 일때

In [6]:
import numpy as np
from collections import deque

# ==================== 설정 ====================
num_frames = 60
np.random.seed(42)

# 시뮬레이션 데이터: 프레임 20부터 화재 시작
confidences = np.concatenate([
    np.random.uniform(0.35, 0.50, 20),      # 정상 
    #'''0.35: 정상 상황에서 나올 수 있는 최소 confidence (아주 낮음)
    #   0.52: 정상 상황에서 나올 수 있는 최대 confidence (아직 0.55 미만)
    #   20: 정상 상황 프레임 개수 → 프레임 0번부터 19번까지'''
    np.linspace(0.58, 0.96, 40)             # 화재 증가
    #'''0.58화재가 시작되는 순간(프레임 20)의 시작 confidence (감지 문턱 바로 위)
    #   0.96화재가 가장 강렬해졌을 때(프레임 59)의 최대 confidence
    #   40화재 상황 프레임 개수 → 프레임 20번부터 59번까지 (총 40프레임)'''
])

# window_size=5 전용 최적 파라미터
p = {'warning': 0.55, 'critical': 0.80, 'min_conf': 0.55, 'alpha': 0.40}

def get_status(vote_ratio, ewma_val):
    if vote_ratio >= p['critical'] and ewma_val >= 0.78: return "🔴 Red"
    elif vote_ratio >= p['warning']: return "⚠️ Orange"
    else: return "✅ Green"

# ==================== 시뮬레이션 실행 ====================
print("=== E-gle Eye window_size=5 단독 시뮬레이션 ===\n")
print("조건: 프레임 20부터 화재 시작 (confidence 점진 증가)\n")
print(f"window_size = 5 | warning={p['warning']:.2f} | critical={p['critical']:.2f} | alpha={p['alpha']:.2f}")
print(f"{'frame':>5} | {'vote%':>6} | {'EWMA':>6} | {'status':>9} | {'conf':>6}")
print("-"*55)

queue = deque(maxlen=5)      # True/False (Fire 여부) 실제 window_size = 5으로 정하는 코드
ewma_val = 0.0

for t in range(num_frames):
    conf = confidences[t]
    queue.append(conf >= 0.55)                    # Fire detected 기준
    
    fire_count = sum(1 for is_fire in queue if is_fire)
    vote_ratio = fire_count / len(queue)
    
    ewma_val = p['alpha'] * conf + (1 - p['alpha']) * ewma_val
    status = get_status(vote_ratio, ewma_val)
    
    # 프레임 15 이후부터 핵심 구간만 출력 (초기 노이즈 제외)
    if t >= 15:
        print(f"{t:5d} | {vote_ratio*100:5.1f}% | {ewma_val:6.3f} | {status:9} | {conf:6.3f}")

print(f"\n→ 최종 상태 (프레임 59 기준): {status}")

=== E-gle Eye window_size=5 단독 시뮬레이션 ===

조건: 프레임 20부터 화재 시작 (confidence 점진 증가)

window_size = 5 | warning=0.55 | critical=0.80 | alpha=0.40
frame |  vote% |   EWMA |    status |   conf
-------------------------------------------------------
   15 |   0.0% |  0.395 | ✅ Green   |  0.378
   16 |   0.0% |  0.395 | ✅ Green   |  0.396
   17 |   0.0% |  0.409 | ✅ Green   |  0.429
   18 |   0.0% |  0.411 | ✅ Green   |  0.415
   19 |   0.0% |  0.404 | ✅ Green   |  0.394
   20 |  20.0% |  0.474 | ✅ Green   |  0.580
   21 |  40.0% |  0.521 | ✅ Green   |  0.590
   22 |  60.0% |  0.552 | ⚠️ Orange |  0.599
   23 |  80.0% |  0.575 | ⚠️ Orange |  0.609
   24 | 100.0% |  0.593 | ⚠️ Orange |  0.619
   25 | 100.0% |  0.607 | ⚠️ Orange |  0.629
   26 | 100.0% |  0.620 | ⚠️ Orange |  0.638
   27 | 100.0% |  0.631 | ⚠️ Orange |  0.648
   28 | 100.0% |  0.642 | ⚠️ Orange |  0.658
   29 | 100.0% |  0.652 | ⚠️ Orange |  0.668
   30 | 100.0% |  0.662 | ⚠️ Orange |  0.677
   31 | 100.0% |  0.672 | ⚠️ Orange | 

# window_size = 7 일때

In [7]:
import numpy as np
from collections import deque

# ==================== 설정 ====================
num_frames = 60
np.random.seed(42)

# 시뮬레이션 데이터: 프레임 20부터 화재 시작
confidences = np.concatenate([
    np.random.uniform(0.35, 0.50, 20),      # 정상
    np.linspace(0.58, 0.96, 40)             # 화재 증가
])

# window_size=7 전용 최적 파라미터
p = {
    'warning': 0.57,      # 4/7 ≈ 57% 이상 → Orange
    'critical': 0.80,     # 6/7 ≈ 86% 이상 → Red
    'min_conf': 0.57,
    'alpha': 0.37
}

def get_status(vote_ratio, ewma_val):
    if vote_ratio >= p['critical'] and ewma_val >= 0.78:
        return "🔴 Red"
    elif vote_ratio >= p['warning']:
        return "⚠️ Orange"
    else:
        return "✅ Green"

# ==================== 시뮬레이션 실행 ====================
print("=== E-gle Eye window_size=7 단독 시뮬레이션 ===\n")
print("조건: 프레임 20부터 화재 시작 (confidence 점진 증가)\n")
print(f"window_size = 7 | warning={p['warning']:.2f} | critical={p['critical']:.2f} | alpha={p['alpha']:.2f}")
print(f"{'frame':>5} | {'vote%':>6} | {'EWMA':>6} | {'status':>9} | {'conf':>6}")
print("-"*55)

queue = deque(maxlen=7)      # True/False (Fire 여부)이자 실제 window_size = 7으로 정하는 코드
ewma_val = 0.0

for t in range(num_frames):
    conf = confidences[t]
    queue.append(conf >= 0.55)                    # Fire detected 기준
    
    fire_count = sum(1 for is_fire in queue if is_fire)
    vote_ratio = fire_count / len(queue)
    
    ewma_val = p['alpha'] * conf + (1 - p['alpha']) * ewma_val
    status = get_status(vote_ratio, ewma_val)
    
    # 프레임 15 이후부터 핵심 구간만 출력 (초기 노이즈 제외)
    if t >= 15:
        print(f"{t:5d} | {vote_ratio*100:5.1f}% | {ewma_val:6.3f} | {status:9} | {conf:6.3f}")

print(f"\n→ 최종 상태 (프레임 59 기준): {status}")

=== E-gle Eye window_size=7 단독 시뮬레이션 ===

조건: 프레임 20부터 화재 시작 (confidence 점진 증가)

window_size = 7 | warning=0.57 | critical=0.80 | alpha=0.37
frame |  vote% |   EWMA |    status |   conf
-------------------------------------------------------
   15 |   0.0% |  0.397 | ✅ Green   |  0.378
   16 |   0.0% |  0.396 | ✅ Green   |  0.396
   17 |   0.0% |  0.408 | ✅ Green   |  0.429
   18 |   0.0% |  0.411 | ✅ Green   |  0.415
   19 |   0.0% |  0.404 | ✅ Green   |  0.394
   20 |  14.3% |  0.469 | ✅ Green   |  0.580
   21 |  28.6% |  0.514 | ✅ Green   |  0.590
   22 |  42.9% |  0.546 | ✅ Green   |  0.599
   23 |  57.1% |  0.569 | ⚠️ Orange |  0.609
   24 |  71.4% |  0.588 | ⚠️ Orange |  0.619
   25 |  85.7% |  0.603 | ⚠️ Orange |  0.629
   26 | 100.0% |  0.616 | ⚠️ Orange |  0.638
   27 | 100.0% |  0.628 | ⚠️ Orange |  0.648
   28 | 100.0% |  0.639 | ⚠️ Orange |  0.658
   29 | 100.0% |  0.650 | ⚠️ Orange |  0.668
   30 | 100.0% |  0.660 | ⚠️ Orange |  0.677
   31 | 100.0% |  0.670 | ⚠️ Orange | 

# window_size = 10 일때

In [8]:
import numpy as np
from collections import deque

# ==================== 설정 ====================
num_frames = 60
np.random.seed(42)

# 시뮬레이션 데이터: 프레임 20부터 화재 시작
confidences = np.concatenate([
    np.random.uniform(0.35, 0.50, 20),      # 정상
    np.linspace(0.58, 0.96, 40)             # 화재 증가
])

# window_size=10 전용 최적 파라미터
p = {
    'warning': 0.60,      # 6/10 = 60% 이상 → Orange
    'critical': 0.80,     # 8/10 = 80% 이상 → Red
    'min_conf': 0.58,
    'alpha': 0.35
}

def get_status(vote_ratio, ewma_val):
    if vote_ratio >= p['critical'] and ewma_val >= 0.78:
        return "🔴 Red"
    elif vote_ratio >= p['warning']:
        return "⚠️ Orange"
    else:
        return "✅ Green"

# ==================== 시뮬레이션 실행 ====================
print("=== E-gle Eye window_size=10 단독 시뮬레이션 ===\n")
print("조건: 프레임 20부터 화재 시작 (confidence 점진 증가)\n")
print(f"window_size = 10 | warning={p['warning']:.2f} | critical={p['critical']:.2f} | alpha={p['alpha']:.2f}")
print(f"{'frame':>5} | {'vote%':>6} | {'EWMA':>6} | {'status':>9} | {'conf':>6}")
print("-"*55)

queue = deque(maxlen=10)     # True/False (Fire 여부)
ewma_val = 0.0

for t in range(num_frames):
    conf = confidences[t]
    queue.append(conf >= 0.55)                    # Fire detected 기준, 실제 window_size = 10으로 정하는 코드
    
    fire_count = sum(1 for is_fire in queue if is_fire)
    vote_ratio = fire_count / len(queue)
    
    ewma_val = p['alpha'] * conf + (1 - p['alpha']) * ewma_val
    status = get_status(vote_ratio, ewma_val)
    
    # 프레임 15 이후부터 핵심 구간만 출력 (초기 노이즈 제외)
    if t >= 15:
        print(f"{t:5d} | {vote_ratio*100:5.1f}% | {ewma_val:6.3f} | {status:9} | {conf:6.3f}")

print(f"\n→ 최종 상태 (프레임 59 기준): {status}")

=== E-gle Eye window_size=10 단독 시뮬레이션 ===

조건: 프레임 20부터 화재 시작 (confidence 점진 증가)

window_size = 10 | warning=0.60 | critical=0.80 | alpha=0.35
frame |  vote% |   EWMA |    status |   conf
-------------------------------------------------------
   15 |   0.0% |  0.398 | ✅ Green   |  0.378
   16 |   0.0% |  0.397 | ✅ Green   |  0.396
   17 |   0.0% |  0.408 | ✅ Green   |  0.429
   18 |   0.0% |  0.410 | ✅ Green   |  0.415
   19 |   0.0% |  0.405 | ✅ Green   |  0.394
   20 |  10.0% |  0.466 | ✅ Green   |  0.580
   21 |  20.0% |  0.509 | ✅ Green   |  0.590
   22 |  30.0% |  0.541 | ✅ Green   |  0.599
   23 |  40.0% |  0.565 | ✅ Green   |  0.609
   24 |  50.0% |  0.584 | ✅ Green   |  0.619
   25 |  60.0% |  0.599 | ⚠️ Orange |  0.629
   26 |  70.0% |  0.613 | ⚠️ Orange |  0.638
   27 |  80.0% |  0.625 | ⚠️ Orange |  0.648
   28 |  90.0% |  0.637 | ⚠️ Orange |  0.658
   29 | 100.0% |  0.648 | ⚠️ Orange |  0.668
   30 | 100.0% |  0.658 | ⚠️ Orange |  0.677
   31 | 100.0% |  0.668 | ⚠️ Orange 